In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt

## 获取图片名称

In [2]:
pic_name_list=os.listdir('data')
pic_name_list

['0.jpg',
 '1.jpg',
 '2.jpg',
 '3.jpg',
 '4.jpg',
 '5.jpg',
 '6.jpg',
 '7.jpg',
 '8.jpg',
 '9.jpg']

## 图片预处理

### 定义图片裁剪函数

In [3]:
def pic_cut(pic_name):
    pic=plt.imread(f'data/{pic_name}')
    N,M,D=pic.shape
    Npic_=pic[135:2825,150:1950]#
    return Npic_

### 定义图片分割函数

In [4]:
def pic_split(Npic):
    pic_data_=[]
    M,N,D=Npic.shape
    for i in range(5):
        for j in range(10):
            pic_new=Npic[270*j:(270*(j+1)),360*i:(360*(i+1)),:]
            pic_data_.append(pic_new)
    return pic_data_

### 颜色矩识别

In [5]:
# 求颜色通道的三阶颜色矩
def var(rd):
    mid=np.mean((rd-rd.mean())**3)
    return np.sign(mid)*abs(mid)**(1/3) #sign取正负号


In [6]:
# 定义数据存放空间
data=[]
labels=[]
for pic_name in pic_name_list:
    #提取图片标签信息
    label=int(pic_name.replace('.jpg',''))
    #对图片进行裁剪
    Npic=pic_cut(pic_name)
    #对图片进行分割
    pic_set=pic_split(Npic)
    for pic_s in pic_set:
        rd=pic_s[:,:,0]
        gd=pic_s[:,:,1]
        bd=pic_s[:,:,2]
        data.append([rd.mean(),gd.mean(),bd.mean(),
                     rd.std(),gd.std(),bd.std(),
                     var(rd), var(gd), var(bd)])
        #记录标签信息
        labels.append(label)

### 数据转换

In [7]:
data=np.array(data)
labels=np.array(labels)

In [8]:
#pip install scikit-learn

### 数据划分

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report

In [10]:
#将样本划分为训练集与测试集
data_tr,data_te,labels_tr,label_te=train_test_split(data,labels,train_size=0.8)


## 训练模型并预测

In [11]:
dtc=DecisionTreeClassifier().fit(data_tr,labels_tr)
pre=dtc.predict(data_te)
sum(pre==label_te)/len(pre)
confusion_matrix(pre,label_te)
print(classification_report(pre,label_te))

              precision    recall  f1-score   support

           0       0.15      0.40      0.22         5
           1       0.88      0.78      0.82         9
           2       0.88      0.70      0.78        10
           3       0.50      0.60      0.55        10
           4       0.75      0.60      0.67        10
           5       0.56      0.42      0.48        12
           6       0.70      0.37      0.48        19
           7       0.33      0.60      0.43         5
           8       0.73      0.73      0.73        11
           9       0.75      1.00      0.86         9

    accuracy                           0.60       100
   macro avg       0.62      0.62      0.60       100
weighted avg       0.66      0.60      0.61       100



# 基于BP神经网络实现

## 图片预处理

In [12]:
#pip install opencv-python

In [13]:
import cv2

In [14]:
# 定义数据存放空间
data=[]
labels=[]
for pic_name in pic_name_list:
    #提取图片标签信息
    label=int(pic_name.replace('.jpg',''))
    #对图片进行裁剪
    Npic=pic_cut(pic_name)
    #对图片进行分割
    pic_set=pic_split(Npic)
    for pic_s in pic_set:
        #对裁剪图片灰度化处理
        gray_pic=cv2.cvtColor(pic_s,cv2.COLOR_BGR2GRAY)
        #对图片进行二值化处理
        #127阈值，像素低于127设置为0（黑色），高于或等于127设置为255（百）
        _,binary_pic=cv2.threshold(gray_pic,127,255,cv2.THRESH_BINARY)
        #修改图片尺寸
        pic_resize=cv2.resize(binary_pic,(64,64))/255
        #记录图片数据
        data.append(pic_resize)
        #记录图片标签
        labels.append(label)

In [15]:
#数据转换（只有数组才能转换成tensor）
data=np.array(data)
labels=np.array(labels)

In [16]:
#将样本划分为训练集与测试集
data_tr,data_te,labels_tr,labels_te=train_test_split(data,labels,train_size=0.8)

## 建立BP神经网络并训练

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset,DataLoader

In [18]:
# 利用类自定义网络
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        #全连接层，可修改
        self.fc1=torch.nn.Linear(64*64,500)
        self.fc2=torch.nn.Linear(500,128)
        self.fc3=torch.nn.Linear(128,64)
        self.fc4=torch.nn.Linear(64,10)    #输出为分类数
        
        
    #向前传播，输入数据为图片处理后张量
    def forward(self,x):
        x=torch.nn.functional.relu(self.fc1(x))
        x=torch.nn.functional.relu(self.fc2(x))
        x=torch.nn.functional.relu(self.fc3(x))
        x=self.fc4(x)
        return x

In [19]:
# 实例化模型
model=Net()
print(model)

Net(
  (fc1): Linear(in_features=4096, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


## 定义损失函数和优化器

In [20]:

# 损失函数
criterion=nn.CrossEntropyLoss()
# 优化器
optimizer=optim.Adam(model.parameters(),lr=0.001)

### 数据转换

In [21]:
# 将data和label从ndarray转换成模型能接受的张量
data_tr_tensor=torch.tensor(data_tr).float()
data_te_tensor=torch.tensor(data_te).float()
labels_tr_tensor=torch.tensor(labels_tr).long()
labels_te_tensor=torch.tensor(labels_te).long()

In [22]:
# 创建tensorDataset对象
train_dataset=TensorDataset(data_tr_tensor,labels_tr_tensor)
test_dataset=TensorDataset(data_te_tensor,labels_te_tensor)


In [23]:
# 创建DataLoader对象  batch_size一次放入样本数量，shuffle打乱样本顺序，提供模型泛化能力
train_loader=DataLoader(train_dataset,batch_size=30,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=30,shuffle=True)

## 定义函数去评估模型效果

In [24]:
def evaluate(test_data,net):
    n_correct=0
    n_total=0
    #禁止梯度回传
    with torch.no_grad():
        for (x,y) in test_data:  #x(batch_size,channels,height,weight)30*1*64*64
            output=net.forward(x.view(-1,64*64))  #-1自动计算该纬度的大小，保持数据元
            for i,output in enumerate(output):
                if torch.argmax(output)==y[i]:#取得概率最大的标签值
                    n_correct+=1
                n_total+=1
    return n_correct/n_total
                

### 训练模型

In [25]:
for i in range(61):
    for batch_idx,(data,label) in enumerate(train_loader):
        #清空梯度
        optimizer.zero_grad()
        
        data=data.view(-1,64*64)
        outputs=model(data)
        loss=criterion(outputs,label)
        
        loss.backward()
        
        optimizer.step()
        
    current_accuracy=evaluate(test_loader,model)
    if i % 10 ==0:
        print(f'Epoch {i+1} accuray: {current_accuracy:.4f}')

Epoch 1 accuray: 0.1100
Epoch 11 accuray: 0.2400
Epoch 21 accuray: 0.3300
Epoch 31 accuray: 0.4400
Epoch 41 accuray: 0.5000
Epoch 51 accuray: 0.4700
Epoch 61 accuray: 0.5000


### 预测数据

In [26]:
label_pre=torch.argmax(model(data_te_tensor.view(-1,64*64)),dim=1)
print(label_pre)
print(labels_te_tensor)

tensor([5, 5, 3, 3, 6, 9, 8, 7, 9, 4, 3, 9, 0, 9, 4, 3, 0, 4, 2, 2, 1, 3, 5, 3,
        5, 0, 8, 3, 5, 2, 5, 1, 5, 3, 0, 8, 3, 5, 5, 5, 3, 5, 7, 6, 5, 2, 3, 5,
        0, 2, 9, 4, 3, 3, 5, 9, 4, 5, 8, 8, 1, 9, 5, 7, 5, 7, 2, 2, 1, 9, 1, 5,
        8, 2, 5, 3, 5, 2, 2, 1, 9, 5, 2, 8, 5, 7, 0, 3, 1, 2, 3, 1, 1, 6, 8, 3,
        1, 5, 9, 9])
tensor([5, 5, 7, 6, 6, 5, 8, 7, 9, 4, 8, 9, 6, 8, 4, 2, 0, 4, 0, 2, 1, 0, 5, 1,
        8, 1, 8, 3, 5, 2, 5, 9, 5, 8, 6, 8, 0, 7, 8, 6, 2, 0, 7, 6, 2, 0, 3, 0,
        0, 2, 8, 4, 5, 6, 9, 9, 4, 0, 9, 8, 1, 7, 9, 7, 5, 7, 2, 4, 7, 3, 1, 5,
        9, 2, 0, 3, 5, 2, 6, 1, 9, 8, 0, 8, 2, 7, 6, 6, 7, 4, 1, 1, 7, 6, 8, 2,
        7, 5, 4, 9])


In [27]:
# 计算精度
sum(labels_te_tensor==label_pre)/len(label_pre)

#混淆矩阵
confusion_matrix(label_pre,labels_te_tensor)
#分类性能报告
print(classification_report(label_pre,labels_te_tensor))

              precision    recall  f1-score   support

           0       0.18      0.33      0.24         6
           1       0.62      0.50      0.56        10
           2       0.55      0.50      0.52        12
           3       0.75      0.18      0.29        17
           4       0.62      1.00      0.77         5
           5       0.83      0.43      0.57        23
           6       0.27      1.00      0.43         3
           7       0.42      1.00      0.59         5
           8       0.46      0.75      0.57         8
           9       0.50      0.45      0.48        11

    accuracy                           0.50       100
   macro avg       0.52      0.61      0.50       100
weighted avg       0.61      0.50      0.49       100



# 基于卷积神经网络

## 模型构建

In [28]:
class Net_CNN(nn.Module):
    def __init__(self):
        super(Net_CNN,self).__init__()
        #卷积层：输入通道数数、输出通道数、卷积核的大小、输入数据的边缘添加的零填充量的数量
        
        self.conv1=nn.Conv2d(1,16,kernel_size=3,padding=1)
        #池化层: 池化窗口大小，步长：池化窗口移动间隔
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
        self.conv2=nn.Conv2d(16,32,kernel_size=3,padding=1)
        self.conv3=nn.Conv2d(32,64,kernel_size=3)
        self.conv4=nn.Conv2d(64,64,kernel_size=3)
        # output=(input-kerner_size+2*padding)/stride+1
        #全连接层
        self.fc1=nn.Linear(256,64)
        self.fc2=nn.Linear(64,10)
        
    def forward(self,x):
        x=self.pool(nn.functional.relu(self.conv1(x)))
        x=self.pool(nn.functional.relu(self.conv2(x)))
        x=self.pool(nn.functional.relu(self.conv3(x)))
        x=self.pool(nn.functional.relu(self.conv4(x)))
        x=x.view(-1,256)
        x=nn.functional.relu(self.fc1(x))
        x=self.fc2(x)
        return x
    

### 实例化

In [29]:
model=Net_CNN()
print(model)

Net_CNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


In [30]:
#损失函数
criterion=nn.CrossEntropyLoss()
#优化器
optimizer=optim.Adam(model.parameters(),lr=0.001)

### 数据转换

In [31]:
#卷积神经网络的输入为（batch_size,channels,size,size），因此需要增加一个维度channels
data_tr_tensor=torch.tensor(np.expand_dims(data_tr,1)).float()
data_te_tensor=torch.tensor(np.expand_dims(data_te,1)).float()
labels_tr_tensor=torch.tensor(labels_tr).long()
labels_te_tensor=torch.tensor(labels_te).long()

In [38]:
data_tr_tensor

tensor([[[[1.0000, 1.0000, 1.0000,  ..., 0.0000, 0.0000, 0.4196],
          [0.8275, 0.8275, 0.8275,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 0.1725, 0.3922, 0.3922]]],


        [[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          ...,
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 0.7725,  ..., 0.5922, 0.5922, 0.5922],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000]]],


        [[[1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000],
          [1.0000, 1.0000, 1.0000,  ..

In [32]:
# 创建tensorDataset对象
train_dataset=TensorDataset(data_tr_tensor,labels_tr_tensor)
test_dataset=TensorDataset(data_te_tensor,labels_te_tensor)
# 创建DataLoader对象  batch_size一次放入样本数量，shuffle打乱样本顺序，提供模型泛化能力
train_loader=DataLoader(train_dataset,batch_size=15,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=15,shuffle=True)

In [33]:
# 检查输入的数据是否符合卷积神经网络的输入要求
for batch_idx,(data,label) in enumerate(train_loader):
    print(data.size(),label.size())

torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size

In [34]:
for batch_idx,(data,label) in enumerate(test_loader):
    print(data.size(),label.size())

torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([15, 1, 64, 64]) torch.Size([15])
torch.Size([10, 1, 64, 64]) torch.Size([10])


### 评估函数

In [35]:
def evaluate(test_data,net):
    n_correct=0
    n_total=0
    #禁止梯度回传
    with torch.no_grad():
        for (x,y) in test_data:  #x(batch_size,channels,height,weight)30*1*64*64
            output=net.forward(x)  #-1自动计算该纬度的大小，保持数据元
            for i,output in enumerate(output):
                if torch.argmax(output)==y[i]:#取得概率最大的标签值
                    n_correct+=1
                n_total+=1
    return n_correct/n_total

In [36]:
for i in range(61):
    for batch_idx,(data,label) in enumerate(train_loader):
        
        optimizer.zero_grad()
        
        outputs=model(data)
        loss=criterion(outputs,label)
        #反向传播
        loss.backward()
        #更新参数
        optimizer.step()
        
    current_accuracy=evaluate(test_loader,model)
    if i%10==0:
        print(f'Epoch {i+1} accuray:{current_accuracy:.4f}')

Epoch 1 accuray:0.1100
Epoch 11 accuray:0.8400
Epoch 21 accuray:0.9400
Epoch 31 accuray:0.9700
Epoch 41 accuray:0.9600
Epoch 51 accuray:0.9700
Epoch 61 accuray:0.9700


## 模型预测

In [39]:
labels_pre=torch.argmax(model(data_te_tensor),dim=1)
print(label_pre)
print(labels_te_tensor)

tensor([5, 5, 7, 6, 6, 5, 8, 7, 9, 4, 8, 9, 6, 9, 0, 2, 0, 4, 0, 2, 1, 0, 5, 1,
        8, 1, 8, 3, 5, 2, 5, 9, 9, 8, 6, 8, 0, 7, 8, 6, 2, 0, 7, 6, 2, 0, 3, 0,
        0, 2, 8, 4, 5, 6, 9, 9, 4, 0, 9, 8, 1, 7, 9, 7, 5, 7, 2, 4, 7, 3, 1, 5,
        9, 2, 0, 3, 5, 2, 6, 1, 9, 8, 0, 8, 2, 7, 6, 6, 7, 4, 1, 1, 7, 6, 8, 2,
        7, 5, 4, 9])
tensor([5, 5, 7, 6, 6, 5, 8, 7, 9, 4, 8, 9, 6, 8, 4, 2, 0, 4, 0, 2, 1, 0, 5, 1,
        8, 1, 8, 3, 5, 2, 5, 9, 5, 8, 6, 8, 0, 7, 8, 6, 2, 0, 7, 6, 2, 0, 3, 0,
        0, 2, 8, 4, 5, 6, 9, 9, 4, 0, 9, 8, 1, 7, 9, 7, 5, 7, 2, 4, 7, 3, 1, 5,
        9, 2, 0, 3, 5, 2, 6, 1, 9, 8, 0, 8, 2, 7, 6, 6, 7, 4, 1, 1, 7, 6, 8, 2,
        7, 5, 4, 9])


In [40]:
# 计算精度
sum(label_pre==labels_te_tensor)/len(label_pre)
#混淆矩阵
confusion_matrix(label_pre,labels_te_tensor)
#分类性能报告
print(classification_report(labels_pre,labels_te_tensor))

              precision    recall  f1-score   support

           0       1.00      0.92      0.96        12
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00        11
           3       1.00      1.00      1.00         4
           4       0.88      1.00      0.93         7
           5       0.92      1.00      0.96        11
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        12
           8       0.92      1.00      0.96        12
           9       1.00      0.83      0.91        12

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100

